In [23]:
import numpy as np
import matplotlib.pyplot as plt
import os
from skopt.space import Real, Integer
from skopt import Optimizer
from skopt.utils import dump, load

First define the dimensions: polymer length and the umbrella

CV space sampled continuously from 1.6 to 6.6 nm COM separation

In [24]:
dim1 = Integer(name='Polymer length', low=0, high=25)
dim2 = Real(name='CV', low=1.6, high=6.6)

Loading the optimizer

In [25]:
optimizer = Optimizer(dimensions=[dim1, dim2],
    base_estimator="gp",
    n_random_starts=0,
    n_initial_points=0,
    n_jobs=3,
    acq_func="EI",
    acq_optimizer="lbfgs",
    random_state=1999)

First load initial data into the model

In [26]:
ini_points = [[0,1.6],
              [0,2.1],
              [0,2.6],
              [0,3.1],
              [0,3.6],
              [0,4.1],
              [0,4.6],
              [0,5.1],
              [0,5.6],
              [0,6.1],
              [0,6.6],
              [10,1.6],
              [10,2.1],
              [10,2.6],
              [10,3.1],
              [10,3.6],
              [10,4.1],
              [10,4.6],
              [10,5.1],
              [10,5.6],
              [10,6.1],
              [10,6.6],
              [20,1.6],
              [20,2.1],
              [20,2.6],
              [20,3.1],
              [20,3.6],
              [20,4.1],
              [20,4.6],
              [20,5.1],
              [20,5.6],
              [20,6.1],
              [20,6.6]
              ]

ini_data=[-30.268820789473633,
           14.579715052631622,
           19.190460644736888,   
           10.33424946052636,
           14.856456881578996,
           21.62012953947351,
           13.628588289473507,
           0.012775631578769259,
           0.3406779078945591,
           12.508910973684033,
           0.21452071052613803,
           -25.188872381578904,
           16.960575907894782,
           21.57553236842109,
           48.089532302631625,
           -3.7394148421052193,
           0.7589639605261377,
           -0.356555881579125,
           0.9089245263156126,
           0.22910882894719065,
           0.7744599078945595,
           0.34886796052613767,
           -29.811372671052588,
           15.544790342105308,
           19.071600368421098,
           11.658016315789519,
           6.284280171052677,
           11.020756381578769,
           -1.3047369078949158,
           25.45458977631561,
           0.0019442236840332768,
           0.3854878947366644,
           -0.7639657763159674]

optimizer.tell(x=ini_points,y=ini_data)

          fun: -30.268820789473633
            x: [0, 1.6]
    func_vals: [-3.027e+01  1.458e+01 ...  3.855e-01 -7.640e-01]
      x_iters: [[0, 1.6], [0, 2.1], [0, 2.6], [0, 3.1], [0, 3.6], [0, 4.1], [0, 4.6], [0, 5.1], [0, 5.6], [0, 6.1], [0, 6.6], [10, 1.6], [10, 2.1], [10, 2.6], [10, 3.1], [10, 3.6], [10, 4.1], [10, 4.6], [10, 5.1], [10, 5.6], [10, 6.1], [10, 6.6], [20, 1.6], [20, 2.1], [20, 2.6], [20, 3.1], [20, 3.6], [20, 4.1], [20, 4.6], [20, 5.1], [20, 5.6], [20, 6.1], [20, 6.6]]
       models: [GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                                        n_restarts_optimizer=2, noise='gaussian',
                                        normalize_y=True, random_state=1393803523)]
        space: Space([Integer(low=0, high=25, prior='uniform', transform='normalize'),
                      Real(low=1.6, high=6.6, prior='uniform', transform='normalize')])
 random_state: RandomState(MT19937)
        sp

Start optimization with the ask-tell interface

Use parallelization so that a few points are sampled at the same time?

In [ ]:
# loading of a previous state
load()

In [18]:
x1 = optimizer.ask()

In [22]:
checked_lengths = set([x[0] for x in optimizer.Xi])
print(checked_lengths)
optimizer.tell(x1, 2.565)
checked_lengths = set([x[0] for x in optimizer.Xi])
print(checked_lengths)

{0, 10, 20}
{0, 10, 3, 20}


In [34]:
a = ['ps', 20]
cmd = f"touch WAIT{a[0]}{a[1]}"
os.system(cmd)

0

In [ ]:
for i in range(3):
    lengths_to_check = []
    x1 = optimizer.ask()
    x2 = optimizer.ask()
    x3 = optimizer.ask()
    for point in (x1, x2, x3):
        # check if plastic was already generated; if no then do the simulation
        checked_lengths = set([x[0] for x in optimizer.Xi])
        if point[0] in checked_lengths:
            lengths_to_check.append(point[0])
            continue

        # check if there is a plastic of the same length in the batch for which the equilibration was already set up
        elif point[0] in lengths_to_check:
            lengths_to_check.append(point[0])
            continue

        # finally, if none of the above are true, carry out the simulation
        # create an empty file as a flag that we must wait for the simulation to finish before proceeding
        else:
            lengths_to_check.append(point[0])
            cmd = f"touch WAIT{optimizer[0]}{optimizer[1]}.tmp"
            os.touch(cmd)
            cmd = f"./gen.sh -p ps -l {point[0]}"
            os.system(cmd)
            #do sbatch here? or maybe in the script directly?

    # what to do when the simulation finishes? how to check for that? remember about the current working directory!!
    # some while loop?
    # instructions what to do when the simulation is finished (put protein and plastic together, simulate them)

    # check if there are no flags left



    # check if plastic already was generated; if yes then skip the next 4 steps
    # gen.sh - plastic generation and preparation for simulation
    # sbatch - submit 50ns plastic equilibration; wait for dry.gro file
    # put_together.py - combine the structure of equilibrated plastic and the protein
    # protplastprep.sh - prepare protein+plastic for equilibration
    # sbatch - submit 100 ns protein+plastic equilibration; wait for dry.gro file (or md.tpr)
    # prep_umb.sh - prepare the system for biased sampling run
    # sbatch - submit 75 ns biased sampling; wait for COLVAR file
    # y = get_force.py - calculate the force from the COLVAR file
    optimizer.tell(x, y)
    dump(res=Optimizer, filename=f"state_{i}.cpt")

In [33]:
os.getcwd()

'/home/fkopczynski/project/notebooks'